In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ST/stargan

Mounted at /content/drive
/content/drive/MyDrive/ST/stargan


# Configuration

In [2]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
import os
import csv
import random
import copy

seed = 2710
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [3]:
config = {
    'realworld': {
        'dataset_name': 'realworld',
        'num_df_domains': 10,
        'num_dp_domains': 5,
        'num_classes': 4,
        'class_names': ['WAL', 'RUN', 'CLD', 'CLU'],
        'num_timesteps': 128,
        'num_channels': 3,
        'num_classes': 4,
    },
    'cwru': {
        'dataset_name': 'cwru_256_3ch_5cl',
        'num_df_domains': 4,
        'num_dp_domains': 4,
        'num_classes': 5,
        'class_names': ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite'],
        'num_timesteps': 256,
        'num_channels': 3,
        'num_classes': 5,
    },
    'realworld_mobiact': {
        'dataset_name': 'realworld_mobiact',
        'num_df_domains': 15,
        'num_dp_domains': 61,
        'num_classes': 4,
        'class_names': ['WAL', 'RUN', 'CLD', 'CLU'],
        'num_timesteps': 128,
        'num_channels': 3,
        'num_classes': 4,
    }
}

syn_name = 'rw05'

In [4]:
def get_data(dataset, domains_set, src_class, domain=None):
    # Load configurations
    dataset_name = config[dataset]['dataset_name']
    class_idx = config[dataset]['class_names'].index(src_class)
    num_df_domains = config[dataset]['num_df_domains']

    try:
        with open(f'data/{dataset_name}.pkl', 'rb') as f:
            x, y, k = pickle.load(f)
        with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
            fs = pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Dataset files for {dataset} not found.")


    if domains_set == 'df':
        mask = (y != class_idx) & (k < num_df_domains) & (fs == 0)
    elif domains_set == 'dp':
        mask = (y != class_idx) & (k >= num_df_domains) & (fs == 0)
    elif domains_set == 'dpfs':
        mask = (y != class_idx) & (k >= num_df_domains) & (fs == 1)
    else:
        raise ValueError(f"Invalid domains set: {domains_set}")

    # Apply initial mask
    x, y, k = x[mask], y[mask], k[mask]

    # Additional domain filtering if specified
    if domain is not None:
        domain_mask = (k == domain)
        x, y, k = x[domain_mask], y[domain_mask], k[domain_mask]

    assert len(x) > 0, f"No data found"
    assert len(x) == len(y) == len(k), f"Data length mismatch"

    return x, y, k



def get_syn_data(dataset, syn_name, src_class, domain=None, fs=False):
    # Load configurations
    class_names = config[dataset]['class_names']

    try:
        with open(f'data/{dataset}_{syn_name}.pkl', 'rb') as f:
            x, y, k = pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Dataset files for {dataset} not found.")

    with open(f'data/{dataset}_fs_{syn_name}.pkl', 'rb') as f:
        fs = pickle.load(f)

    x, y, k = x[fs == 0], y[fs == 0], k[fs == 0]

    if domain is not None:
        domain_mask = (k == domain)
        x, y, k = x[domain_mask], y[domain_mask], k[domain_mask]

    assert len(x) > 0, f"No data found"
    assert len(x) == len(y) == len(k), f"Data length mismatch"

    return x, y, k



class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=5):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs



def remap_labels(y):
    label_map = {clss: i for i, clss in enumerate(np.unique(y))}
    return np.array([label_map[clss] for clss in y])



def get_dataloader(x, y, shuffle=False):
    x = torch.tensor(x, dtype=torch.float32)
    y = remap_labels(y)
    y = torch.tensor(y, dtype=torch.long)

    # Determine dataset size and batch size
    dataset_size = len(x)
    if dataset_size <= 100:
        batch_size = dataset_size
    elif dataset_size <= 1000:
        batch_size = 16
    elif dataset_size <= 5000:
        batch_size = 32
    else:
        batch_size = 64

    dataset = TensorDataset(x, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return loader



def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    loss_fn = nn.CrossEntropyLoss()

    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == y_batch).sum().item()
            total_predictions += len(y_batch)

    total_loss /= len(test_loader)
    accuracy = correct_predictions / total_predictions

    return accuracy, total_loss



def train_model(model, train_loader, val_loader, optimizer, num_epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_fn = nn.CrossEntropyLoss()

    loss_train = []
    loss_val = []
    accuracy_val = []
    best_model_state = None
    best_loss = np.inf
    best_accuracy = 0

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / num_epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(train_loader)
        loss_train.append(total_loss)

        # Update learning rate
        scheduler.step()

        val_accuracy, val_loss = evaluate_model(model, val_loader)
        if val_loss < best_loss:
            best_epoch = epoch
            best_accuracy = val_accuracy
            best_loss = val_loss
            best_model_state = model.state_dict().copy()

        loss_val.append(val_loss)
        accuracy_val.append(val_accuracy)

        current_lr = scheduler.get_last_lr()[0]
        if (epoch+1) % 10 == 0:
            print(f"\tEpoch {epoch + 1}/{num_epochs} - Train loss: {total_loss:.4f} - Val accuracy: {val_accuracy:.4f} - Val loss: {val_loss:.4f} - LR: {current_lr:.2e}")

    print(f"\tBest epoch: {best_epoch + 1} - Best val accuracy: {best_accuracy:.4f} - Best val loss: {best_loss:.4f}\n")

    # Load best model state
    model.load_state_dict(best_model_state)

    return model



def train_and_test(x_train, y_train, x_test, y_test, dataset, num_epochs=100):
    assert np.array_equal(np.unique(y_train), np.unique(y_test)), "Training and test labels do not match"

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)
    test_loader = get_dataloader(x_test, y_test)

    model = TSTRClassifier(num_timesteps=config[dataset]['num_timesteps'],
                           num_channels=config[dataset]['num_channels'],
                           num_classes=config[dataset]['num_classes']-1)
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    test_accuracy, test_loss = evaluate_model(trained_model, test_loader)

    return test_accuracy, test_loss



def train_only(x_train, y_train, dataset, num_epochs=100):

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)

    model = TSTRClassifier(num_timesteps=config[dataset]['num_timesteps'],
                           num_channels=config[dataset]['num_channels'],
                           num_classes=config[dataset]['num_classes']-1)
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    return trained_model



def train_classifier_cv(x_train, y_train, dataset, num_epochs=100):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    accs = []
    losses = []

    for train_index, test_index in skf.split(x_train, y_train):
        x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        acc, loss = train_and_test(x_train_fold, y_train_fold, x_test_fold, y_test_fold, dataset, num_epochs)
        accs.append(acc)
        losses.append(loss)

    return np.mean(accs), np.mean(losses)



def fine_tune(model, x_train, y_train, num_epochs=100):
    # Freeze feature extraction layers
    for name, param in model.named_parameters():
        if 'conv' in name or 'bn' in name:
            param.requires_grad = False

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)

    initial_lr = 0.00001
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    return trained_model



def save_scores(source, domain, accuracy, loss, name, dataset):
    results_dir = 'results'
    # Ensure the directory exists
    os.makedirs(results_dir, exist_ok=True)
    # Path to the CSV file
    file_path = os.path.join(results_dir, f'{dataset}_{name}.csv')
    # Check if the file exists
    file_exists = os.path.exists(file_path)

    # Open the file in append mode if it exists, or write mode if it doesn't
    with open(file_path, mode='a' if file_exists else 'w', newline='') as file:
        writer = csv.writer(file)
        # If the file does not exist, write the header
        if not file_exists:
            writer.writerow(['source', 'domain', 'accuracy', 'loss'])
        # Write the data rows
        writer.writerow([source, domain, accuracy, loss])

# TSTR

In [ ]:
def compute_TSTR_Dp(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train and evaluate via cross-validation on Dp data
            print('Training and evaluating on Dp data via cross-validation...')
            acc, loss = train_classifier_cv(x_dp_dom, y_dp_dom, dataset, num_epochs=100)
            save_scores(src_class, domain, acc, loss, 'Dp', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTR_Dp('realworld')

In [ ]:
def compute_TSTR_Dpc(dataset):

    raise NotImplementedError



# compute_TSTR_Dpc('realworld')

In [ ]:
def compute_TSTR_Df(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=100)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Evaluate on Dp data
            acc, loss = evaluate_model(df_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'Df', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTR_Df('realworld')

In [ ]:
def compute_TSTR_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data and evaluate on Dp data
            print('Training on synthetic data...')
            acc, loss = train_and_test(x_syn_dom, y_syn_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=100)
            save_scores(src_class, domain, acc, loss, 'Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTR_Syn('realworld')

In [ ]:
def compute_TSTR_Df_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=100)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on synthetic data and evaluate on Dp data
            print('Fine-tuning on synthetic data...')
            df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn_dom, y_syn_dom, num_epochs=100)
            acc, loss = evaluate_model(df_syn_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'Df_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTR_Df_Syn('realworld')

# TSTRFS

In [5]:
def compute_TSTRFS_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Dpfs data and test on Dp data
            print('Training on Dpfs data...')
            acc, loss = train_and_test(x_dpfs_dom, y_dpfs_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=100)
            save_scores(src_class, domain, acc, loss, 'FS_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Dpfs('realworld')

Source class: WAL

Domain: 10
x_dpfs_dom.shape: (15, 3, 128) | np.unique(y_dpfs_dom): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on Dpfs data...
	Epoch 10/100 - Train loss: 0.7048 - Val accuracy: 0.3333 - Val loss: 1.1159 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.4177 - Val accuracy: 0.3333 - Val loss: 1.1871 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.2865 - Val accuracy: 0.3333 - Val loss: 1.2996 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.1913 - Val accuracy: 0.3333 - Val loss: 1.2810 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.1344 - Val accuracy: 0.6667 - Val loss: 0.7917 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.1014 - Val accuracy: 1.0000 - Val loss: 0.4562 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0713 - Val accuracy: 1.0000 - Val loss: 0.3926 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0724 - Val accuracy: 1.0000 - Val loss: 0.3924 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0749 - Val accuracy: 1.0000 - Val loss: 0.3959 - LR: 

In [6]:
def compute_TSTRFS_Df_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=100)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on Dpfs data and test on Dp data
            print('Fine-tuning on Dpfs data...')
            df_dpfs_model = fine_tune(copy.deepcopy(df_model), x_dpfs_dom, y_dpfs_dom, num_epochs=100)
            acc, loss = evaluate_model(df_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Df_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Df_Dpfs('realworld')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.0233 - Val accuracy: 0.9830 - Val loss: 0.0540 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0089 - Val accuracy: 0.9839 - Val loss: 0.0501 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0035 - Val accuracy: 0.9839 - Val loss: 0.0427 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0031 - Val accuracy: 0.9858 - Val loss: 0.0436 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0013 - Val accuracy: 0.9858 - Val loss: 0.0424 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0018 - Val accuracy: 0.9858 - Val loss: 0.0509 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0004 - Val accuracy: 0.9877 - Val loss: 0.0451 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0004 - Val accuracy: 0.9849 - Val loss: 0.0481 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0007 - Val accuracy: 0.9849 - Val loss: 0.0526 - LR: 1.00e-05
	Epoch 100/100 - Train loss: 0.0003 - Val accuracy: 0.9839 - Val loss: 0.04

In [7]:
def compute_TSTRFS_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data and evaluate on Dp data
            print('Training on synthetic data...')
            acc, loss = train_and_test(x_syn_dom, y_syn_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=100)
            save_scores(src_class, domain, acc, loss, 'FS_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Syn('realworld')

Source class: WAL

Domain: 10
x_syn_dom.shape: (726, 3, 128) | np.unique(y_syn_dom): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on synthetic data...
	Epoch 10/100 - Train loss: 0.0146 - Val accuracy: 1.0000 - Val loss: 0.0031 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0025 - Val accuracy: 1.0000 - Val loss: 0.0008 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0018 - Val accuracy: 1.0000 - Val loss: 0.0006 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0007 - Val accuracy: 1.0000 - Val loss: 0.0005 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0004 - Val accuracy: 1.0000 - Val loss: 0.0002 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0004 - Val accuracy: 1.0000 - Val loss: 0.0002 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0005 - Val accuracy: 1.0000 - Val loss: 0.0001 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0010 - Val accuracy: 1.0000 - Val loss: 0.0004 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0009 - Val accuracy: 1.0000 - Val loss: 0.0001 - 

In [8]:
def compute_TSTRFS_Df_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=100)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on synthetic data and evaluate on Dp data
            print('Fine-tuning on synthetic data...')
            df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn_dom, y_syn_dom, num_epochs=100)
            acc, loss = evaluate_model(df_syn_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Df_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_Syn('realworld')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.0354 - Val accuracy: 0.9735 - Val loss: 0.0659 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0113 - Val accuracy: 0.9783 - Val loss: 0.0538 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0048 - Val accuracy: 0.9820 - Val loss: 0.0534 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0031 - Val accuracy: 0.9792 - Val loss: 0.0639 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0015 - Val accuracy: 0.9811 - Val loss: 0.0656 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0171 - Val accuracy: 0.9792 - Val loss: 0.0672 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0006 - Val accuracy: 0.9820 - Val loss: 0.0614 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0009 - Val accuracy: 0.9811 - Val loss: 0.0590 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0004 - Val accuracy: 0.9820 - Val loss: 0.0684 - LR: 1.00e-05
	Epoch 100/100 - Train loss: 0.0002 - Val accuracy: 0.9830 - Val loss: 0.06

In [9]:
def compute_TSTRFS_Syn_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data
            print('Training on synthetic data...')
            syn_model = train_only(x_syn_dom, y_syn_dom, dataset, num_epochs=100)

            # Fine-tune synthetic model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            syn_dpfs_model = fine_tune(copy.deepcopy(syn_model), x_dpfs_dom, y_dpfs_dom, num_epochs=100)
            acc, loss = evaluate_model(syn_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Syn_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Syn_Dpfs('realworld')

Source class: WAL

Domain: 10
x_syn_dom.shape: (726, 3, 128) | np.unique(y_syn_dom): [1 2 3]
x_dpfs_dom.shape: (15, 3, 128) | np.unique(y_dpfs_dom): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on synthetic data...
	Epoch 10/100 - Train loss: 0.0171 - Val accuracy: 1.0000 - Val loss: 0.0021 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0059 - Val accuracy: 1.0000 - Val loss: 0.0008 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0014 - Val accuracy: 1.0000 - Val loss: 0.0007 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0104 - Val accuracy: 1.0000 - Val loss: 0.0003 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0011 - Val accuracy: 1.0000 - Val loss: 0.0003 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0045 - Val accuracy: 1.0000 - Val loss: 0.0004 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0005 - Val accuracy: 1.0000 - Val loss: 0.0003 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0007 - Val accuracy: 1.0000 - Val loss: 0.0002 - LR: 2.00e-05
	Epoch 90/100 -

In [10]:
def compute_TSTRFS_Df_Syn_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=100)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on synthetic data
            print('Fine-tuning on synthetic data...')
            df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn_dom, y_syn_dom, num_epochs=100)

            # Fine-tune Df-syn model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            df_syn_dpfs_model = fine_tune(copy.deepcopy(df_syn_model), x_dpfs_dom, y_dpfs_dom, num_epochs=100)
            acc, loss = evaluate_model(df_syn_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Df_Syn_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_Syn_Dpfs('realworld')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.0250 - Val accuracy: 0.9830 - Val loss: 0.0446 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0075 - Val accuracy: 0.9858 - Val loss: 0.0513 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0021 - Val accuracy: 0.9868 - Val loss: 0.0541 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0013 - Val accuracy: 0.9858 - Val loss: 0.0505 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0006 - Val accuracy: 0.9858 - Val loss: 0.0593 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0010 - Val accuracy: 0.9830 - Val loss: 0.0707 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0002 - Val accuracy: 0.9858 - Val loss: 0.0616 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0004 - Val accuracy: 0.9868 - Val loss: 0.0624 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0002 - Val accuracy: 0.9858 - Val loss: 0.0674 - LR: 1.00e-05
	Epoch 100/100 - Train loss: 0.0002 - Val accuracy: 0.9868 - Val loss: 0.06

In [11]:
def compute_TSTRFS_Df_plus_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Join Df and Dpfs data
            x_df_dpfs = np.concatenate([x_df, x_dpfs_dom], axis=0)
            y_df_dpfs = np.concatenate([y_df, y_dpfs_dom], axis=0)
            k_df_dpfs = np.concatenate([k_df, k_dpfs_dom], axis=0)
            print(f'x_df_dpfs.shape: {x_df_dpfs.shape} | np.unique(y_df_dpfs): {np.unique(y_df_dpfs)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Df plus Dpfs data and test on Dp data
            print('Training on Df plus Dpfs data...')
            acc, loss = train_and_test(x_df_dpfs, y_df_dpfs, x_dp_dom, y_dp_dom, dataset, num_epochs=100)
            save_scores(src_class, domain, acc, loss, 'FS_Df_plus_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Df_plus_Dpfs('realworld')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Domain: 10
x_dpfs_dom.shape: (15, 3, 128) | np.unique(y_dpfs_dom): [1 2 3]
x_df_dpfs.shape: (5303, 3, 128) | np.unique(y_df_dpfs): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on Df plus Dpfs data...
	Epoch 10/100 - Train loss: 0.0232 - Val accuracy: 0.9859 - Val loss: 0.0366 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0079 - Val accuracy: 0.9783 - Val loss: 0.0624 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0013 - Val accuracy: 0.9877 - Val loss: 0.0378 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0040 - Val accuracy: 0.9877 - Val loss: 0.0368 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0006 - Val accuracy: 0.9896 - Val loss: 0.0346 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0010 - Val accuracy: 0.9887 - Val loss: 0.0393 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0002 - Val accuracy: 0.9877 - Val loss: 0.0448 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0002 - Val accurac

In [12]:
def compute_TSTRFS_Df_plus_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Join Df and synthetic data
            x_df_syn = np.concatenate([x_df, x_syn_dom], axis=0)
            y_df_syn = np.concatenate([y_df, y_syn_dom], axis=0)
            k_df_syn = np.concatenate([k_df, k_syn_dom], axis=0)
            print(f'x_df_syn.shape: {x_df_syn.shape} | np.unique(y_df_syn): {np.unique(y_df_syn)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Df plus synthetic data and test on Dp data
            print('Training on Df plus synthetic data...')
            acc, loss = train_and_test(x_df, y_df, x_dp_dom, y_dp_dom, dataset, num_epochs=100)
            save_scores(src_class, domain, acc, loss, 'FS_Df_plus_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_plus_Syn('realworld')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Domain: 10
x_syn_dom.shape: (726, 3, 128) | np.unique(y_syn_dom): [1 2 3]
x_df_syn.shape: (6014, 3, 128) | np.unique(y_df_syn): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on Df plus synthetic data...
	Epoch 10/100 - Train loss: 0.0312 - Val accuracy: 0.9792 - Val loss: 0.0470 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0076 - Val accuracy: 0.9868 - Val loss: 0.0421 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0021 - Val accuracy: 0.9849 - Val loss: 0.0451 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0011 - Val accuracy: 0.9887 - Val loss: 0.0438 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0024 - Val accuracy: 0.9811 - Val loss: 0.0598 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0011 - Val accuracy: 0.9830 - Val loss: 0.0597 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0023 - Val accuracy: 0.9754 - Val loss: 0.0803 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0002 - Val accur